<a href="https://colab.research.google.com/github/UiinKim/Music_Generation/blob/main/LSTM_MusicGeneration_%EC%8B%A4%EC%8A%B5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import torchvision
import torch.nn

!pip install --upgrade torch torchvision

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 670.2/670.2 MB 1.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 74.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 61.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 72.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 91.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 11.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.8/209.8 MB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━

In [2]:
torch.cuda.is_available()

True

In [3]:
from google.colab import files
uploaded=files.upload()

Saving av_p323.mid to av_p323.mid
Saving avipf4_3.mid to avipf4_3.mid
Saving avsona55.mid to avsona55.mid
Saving op1-1a.mid to op1-1a.mid
Saving op1-1b.mid to op1-1b.mid
Saving rv-107.mid to rv-107.mid
Saving v1_40077.mid to v1_40077.mid
Saving v2_40077.mid to v2_40077.mid
Saving v3_40077.mid to v3_40077.mid
Saving viv3_1.mid to viv3_1.mid
Saving viv3_2.mid to viv3_2.mid
Saving viv3_3.mid to viv3_3.mid
Saving viv3_4.mid to viv3_4.mid
Saving viv13-31.mid to viv13-31.mid
Saving viv13-32.mid to viv13-32.mid
Saving viv13-33.mid to viv13-33.mid
Saving viv13-34.mid to viv13-34.mid
Saving viv13-35.mid to viv13-35.mid
Saving viva_f.mid to viva_f.mid
Saving viva_f-1.mid to viva_f-1.mid
Saving viva_fi.mid to viva_fi.mid
Saving viva_fid.mid to viva_fid.mid
Saving vivaldi-.mid to vivaldi-.mid
Saving vivaldi3.mid to vivaldi3.mid
Saving vivcfob1.mid to vivcfob1.mid
Saving vivcfob2.mid to vivcfob2.mid
Saving vivcfob3.mid to vivcfob3.mid
Saving vivrv52a.mid to vivrv52a.mid
Saving vivrv52b.mid to vivrv

In [12]:
import numpy as np

In [32]:
def part_check(parts):
  for part in parts.parts:
    print(part)
  return len(parts)

def max_parts(parts):
  lens=[]
  lens.append(part_check(parts))
  return np.max(lens)

def instrument_part_parse(parts_i):
  notes_to_parse=parts_i.recurse() #midi 라는 stream(객체) 안에 존재하는 iterable(순서가 존재할 수 있는 list tuple 등)한 객체(여기선 list)를 iterator로 만든다-> 순서가 생김
  print(notes_to_parse)
  return notes_to_parse

def append_notes(notes_to_parse, notes):
  for element in notes_to_parse:
    if isinstance(element, note.Note): #element가 note인 상태이면
      notes.append(str(element.pitch)) #문자열로 note 추가한다
    elif isinstance(element, chord.Chord): #element가 chord인 상태이면
      notes.append('.'.join(str(n)for n in element.normalOrder)) #문자열로 .으로 구분된 note들이 합해진 chord를 구분하여 note들로 추가

In [40]:
from music21 import converter, instrument, note, chord
#import glob # jupyter 용

notes=[]
max_len=0 #가장 파트가 많은 길이

#for file in glob.glob('C:/Users/UiinKim/MusicGeneration/Classical-Piano-Composer-master/midi_songs/*.mid'):
for file in uploaded:
    midi=converter.parse(file) #converter은 음악을 로드한다. parse를 사용하여 파일의 item을 파싱하고 stream(객체)에 넣어준다
    notes_to_parse=None
    notes_i=[]
    try:
        parts=instrument.partitionByInstrument(midi)#단일 stream, score, multi part 구조인 경우에 각각 악기별로 partition을 나누고 다른 파트들을 하나로 합쳐준다.(피아노로 합한다는 것 같음)
    except TypeError:
        print('## 1 {} file occur error.'.format(file))
    if parts:
        print('## 2 {} file has instrument parts'.format(file))
        if max_len<max_parts(parts):
          max_len=max_parts(parts)
          print(max_len)
        for i in range(len(parts)):
          notes_to_parse=instrument_part_parse(parts.parts[i])
          append_notes(notes_to_parse, notes_i)
          notes.append(notes_i)
    else:
        print('## 3 {} file has notes in a flat structure'.format(file)) #평평한 구조이므로 그냥 추가
        notes_to_parse=midi.flat.notes #평평하게 음표를 넣는 것인듯
        append_notes(notes_to_parse, notes_i)
        notes.append(notes_i)


## 2 av_p323.mid file has instrument parts
<music21.stream.Part Recorder>
<music21.stream.Part Oboe>
<music21.stream.Part Violin>
<music21.stream.Part Bassoon>
<music21.stream.Part Harpsichord>
<music21.stream.Part Violoncello>
<music21.stream.Part Recorder>
<music21.stream.Part Oboe>
<music21.stream.Part Violin>
<music21.stream.Part Bassoon>
<music21.stream.Part Harpsichord>
<music21.stream.Part Violoncello>
6
<music21.stream.iterator.RecursiveIterator for Part:Recorder @:0>
<music21.stream.iterator.RecursiveIterator for Part:Oboe @:0>
<music21.stream.iterator.RecursiveIterator for Part:Violin @:0>
<music21.stream.iterator.RecursiveIterator for Part:Bassoon @:0>
<music21.stream.iterator.RecursiveIterator for Part:Harpsichord @:0>
<music21.stream.iterator.RecursiveIterator for Part:Violoncello @:0>
## 2 avipf4_3.mid file has instrument parts
<music21.stream.Part Viola>
<music21.stream.Part Contrabass>
<music21.stream.iterator.RecursiveIterator for Part:Viola @:0>
<music21.stream.iterat

In [43]:
print(notes)
print(len(notes))
total=[]
for note_i in notes:
  for x in note_i:
    total.append(x)

[['F4', 'A4', 'C5', 'C5', 'A5', 'B-5', 'C6', 'B-5', 'A5', 'F5', 'C5', 'A4', 'C5', 'F5', 'A5', 'B-5', 'C6', 'B-5', 'A5', 'F5', 'C5', 'A4', 'C5', 'F5', 'F5', 'C5', 'A4', 'F4', 'C5', 'C5', 'A5', 'B-5', 'C6', 'B-5', 'B5', 'C6', 'D6', 'C6', 'C#6', 'D6', 'E6', 'D6', 'G5', 'F5', 'E5', 'F5', 'G5', 'F5', 'E5', 'F5', 'C5', 'A4', 'F4', 'F5', 'C5', 'A4', 'F4', 'C5', 'C5', 'F5', 'C5', 'A4', 'F4', 'F5', 'C5', 'A4', 'F4', 'C5', 'C5', 'F4', 'C5', 'F5', 'C5', 'A4', 'C5', 'F5', 'C5', 'A4', 'C5', 'F5', 'G5', 'A5', 'F5', 'C5', 'F5', 'A5', 'F5', 'C5', 'F5', 'A5', 'B-5', 'A5', 'F5', 'C5', 'F5', 'A5', 'F5', 'C5', 'A5', 'B-5', 'A5', 'G5', 'E5', 'E5', 'G5', 'G5', 'A5', 'G5', 'A5', 'G5', 'C5', 'C5', 'F5', 'F5', 'G5', 'A5', 'A5', 'B-5', 'C6', 'C6', 'B-5', 'A5', 'G5', 'F5', 'E-5', 'A5', 'C6', 'E-5', 'D5', 'G5', 'G5', 'A5', 'B5', 'B5', 'C6', 'D6', 'D6', 'C6', 'B5', 'A5', 'G5', 'F5', 'B5', 'D6', 'F5', 'E5', 'A5', 'A5', 'B5', 'C#6', 'C#6', 'D6', 'E6', 'E6', 'D6', 'C#6', 'B5', 'A5', 'G5', 'C#6', 'E6', 'G5', 'F5', 'F5

In [47]:
import numpy as np
from tensorflow.python.keras.utils import np_utils
sequence_length=10 #음표 하나를 예측하는 데 사용하는 음표의 개수
#곡에 있는 모든 계이름의 이름을 pitchnames 변수에 저장 -> set(중복 제거)과 sorted(정렬->음표는 C, D등으로 정렬 가능)로 사전 만들기 준비

pitchnames=sorted(set(item for item in total))
n_vocab=len(pitchnames)

#음표사전을 만든다
note_to_int=dict((note, number) for number, note in enumerate(pitchnames))
network_input=[]
network_output=[]

#입력 시퀀스 만들기
for note_i in notes:
  for i in range(0, len(note_i)-sequence_length, 1): #하나의 note를 생성하는데 100개의 note를 소모하므로 i(생성되는 note)는 1씩 증가하고 마지막 길이는 최대 길이에서 시퀀스의 길이만큼 빼야함
      sequence_in=note_i[i:i+sequence_length] #100개의 note를 사용하여 1개를 예측하고
      sequence_out=note_i[i+sequence_length] #101번째의 note가 예측된 레이블 1개의 note
      network_input.append([note_to_int[char] for char in sequence_in]) #int로 바꿔서 모델에 넣을 input에 100개의 note를 가진 1개의 리스트로 추가한다
      network_output.append(note_to_int[sequence_out])#int로 바꿔서 1개의 note 레이블로 사용

n_patterns=len(network_input) #총 시퀀스의 개수

#데이터 입력 형태를 LSTM 레이어에 알맞게 변경
network_input=np.reshape(network_input, (n_patterns, sequence_length,1))
#괄호 안 설명 -> 입력으로 들어갈 1개의 시퀀스 당 100개의 note로 구성된 network_input을 총 시퀀스의 개수만큼의 note로 출력되고 한 note를 예측하는데(1을 뜻함->한 note) 사용하는 시퀀스의 길이는 100(한 시퀀스의 길이)로 설정한다.
#n_pattern은 1epoch 당 들어가는 배치의 크기(network_input의 총 시퀀스 수만큼 모두 1epoch당 들어간다)
#입력값을 normalizing(정규화)
network_input=network_input/float(n_vocab) #note_input을 음표사전의 길이로 나눈다 -> 정규화를 위함(데이터를 0~1사이로 스케일링 -> 이유는 입력 값이 가능한 모든 음표의 수에 대한 비율로 나누기 때문)-> 일정한 범위 이내에 있으므로 모델의 수렴 속도 향상 및 가중치 업데이트 안정화
network_output=np_utils.to_categorical(network_output) #one-hot encoding 10진->2진

In [48]:
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.layers import Embedding
from keras.layers import LSTM
from keras.layers import Activation, Dense
from keras.callbacks import ModelCheckpoint

model=Sequential()
model.add(LSTM(256, input_shape=(network_input.shape[1], network_input.shape[2]), return_sequences=True))
#이 층에서 가지는 뉴런 수가 256개-> output_dim이다, 100개의 note가 들어오고 1개의 음표사전 중 있는 음을 나타내기 위해 1개의 note를 표현한다(음표의 특성 수), 시퀀스 출력을 반환한다.
model.add(Dropout(0.3)) #죽이는 뉴런 비율
model.add(LSTM(512, return_sequences=True)) #모델의 표현력 향상을 위해 512
model.add(Dropout(0.3))
model.add(LSTM(256)) #return_sequences가 없으므로 출력만 전달(Fully connected를 위함인듯)
model.add(Dense(256)) #Fully connected layer로 LSTM의 출력을 전달받아 추가적인 표현 가능
model.add(Dropout(0.3))
model.add(Dense(n_vocab))#신경망의 출력을 음표의 확률 분포로 변환 -> n_vocab의 이유는 음표 사전에 존재하는 음표들로만 출력이 되야하기 때문
model.add(Activation('softmax')) #음표가 속할 확률을 출력
model.compile(loss='categorical_crossentropy', optimizer='rmsprop')
#두 개 이상의 카테고리 존재 -> -categorical_crossentropy, 순환신경망에서 자주 사용하는 rmsprop

In [49]:
from keras.callbacks import ModelCheckpoint

filepath='weights-improvement-{epoch:02d}-{loss:4f}-bigger.hdf5'
#모델 가중치를 저장할 파일 경로
checkpoint=ModelCheckpoint(filepath, monitor='loss', verbose=0, save_best_only=True, mode='min')
#모델 체크포인트는 매 에포크마다 네트워크 노드의 가중치를 파일에 저장하는 방법, 기준은 손실이고 가장 작은 값의 모델의 구조와 가중치만 저장
callbacks_list=[checkpoint]
model.fit(network_input, network_output, epochs=200, batch_size=64, callbacks=callbacks_list)
#입력 시퀀스, 출력 시퀀스, 200번 반복, 64개의 샘플로 가중치 갱신,

Epoch 1/200
4665/4665 [==============================] - 48s 9ms/step - loss: 3.9672
Epoch 2/200
   6/4665 [..............................] - ETA: 47s - loss: 3.7272

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


4665/4665 [==============================] - 39s 8ms/step - loss: 3.3307
Epoch 3/200
4665/4665 [==============================] - 40s 9ms/step - loss: 2.5837
Epoch 4/200
4665/4665 [==============================] - 40s 9ms/step - loss: 2.0231
Epoch 5/200
4665/4665 [==============================] - 40s 9ms/step - loss: 1.6306
Epoch 6/200
4665/4665 [==============================] - 40s 9ms/step - loss: 1.3460
Epoch 7/200
4665/4665 [==============================] - 41s 9ms/step - loss: 1.1397
Epoch 8/200
4665/4665 [==============================] - 40s 9ms/step - loss: 0.9903
Epoch 9/200
4294/4665 [==========================>...] - ETA: 3s - loss: 0.8843

KeyboardInterrupt: ignored

In [ ]:
import os
print(os.getcwd())  # 현재 작업 디렉토리 확인
# '/content' 디렉토리에 있는 파일 목록 확인
files = os.listdir('/content')
print(files)

/content
['.config', 'z_aeristhemepiano.mid', 'dontbeafraid.mid', 'gerudo.mid', 'weights-improvement-05-3.253520-bigger.hdf5', 'Gold_Silver_Rival_Battle.mid', 'weights-improvement-09-4.191451-bigger.hdf5', 'great_war.mid', 'weights-improvement-47-0.937721-bigger.hdf5', 'thenightmarebegins.mid', 'weights-improvement-15-2.348154-bigger.hdf5', 'rufus.mid', 'weights-improvement-19-2.077695-bigger.hdf5', 'Finalfantasy5gilgameshp.mid', 'relmstheme-piano.mid', 'ff1battp.mid', 'balamb.mid', 'weights-improvement-81-0.608602-bigger.hdf5', 'sobf.mid', 'weights-improvement-44-1.009022-bigger.hdf5', 'weights-improvement-65-0.700844-bigger.hdf5', 'Ff7-Jenova_Absolute.mid', 'weights-improvement-63-0.722145-bigger.hdf5', 'ultimafro.mid', 'weights-improvement-76-0.622068-bigger.hdf5', 'caitsith.mid', 'weights-improvement-80-0.611151-bigger.hdf5', 'weights-improvement-34-1.322837-bigger.hdf5', 'ultros.mid', 'weights-improvement-08-2.944376-bigger.hdf5', '0fithos.mid', 'weights-improvement-60-0.750970-bi

In [ ]:
#음악 생성
model = Sequential()
model.add(LSTM(
    512,
    input_shape=(network_input.shape[1], network_input.shape[2]),
    return_sequences=True
))
model.add(Dropout(0.3))
model.add(LSTM(512, return_sequences=True))
model.add(Dropout(0.3))
model.add(LSTM(512))
model.add(Dense(256))
model.add(Dropout(0.3))
model.add(Dense(n_vocab))
model.add(Activation('softmax'))
model.compile(loss='categorical_crossentropy', optimizer='rmsprop')

# 각각의 뉴런(노드)의 가중치를 로드합니다.
# 파일에 저장한 학습 결과를 가져오는 것과 같습니다!
model.load_weights('weights.hdf5') #가중치 어떤 파일 가져왔는지 물어보기

In [ ]:
# 입력 시퀀스 랜덤
start = np.random.randint(0, len(network_input) - 1)

# 숫자 -> 노트 매핑 사전 생성
int_to_note = dict((number, note) for number, note in enumerate(pitchnames))
pattern = network_input[start]
#random으로 구한 index를 사용하여 input으로 들어간 network의 start번째 시퀀스를 pattern으로 지정한다.

prediction_output = []

# 500개의 note 생성
for note_index in range(500):
    prediction_input = np.array(pattern).reshape(1, len(pattern), 1)
    # LSTM 구조로 변경. 1의 배치사이즈(1 epoch 당 한 개의 note를 나타내는 한 시퀀스(network_input[0])가 들어간다), 한 시퀀스의 길이(100)만큼의 길이가 들어가고, 1개의 note를 나타낸다.
    #결국 1epoch 당 한개의 note를 나타내기 위해 한개의 시퀀스가 들어가는 느낌
    prediction_input = prediction_input / float(n_vocab) #0과 1사이로 표현

    # 입력값에 대해 다음 note 예측
    prediction = model.predict(prediction_input, verbose=0)
    index = np.argmax(prediction)  # 예측값들 중 가장 예측값(softmax 활용)이 높은 인덱스를 선정

    # 결과를 int->note로 변경
    result = int_to_note[index]
    prediction_output.append(result) #output에 string으로 바뀐 예측 note를 저장

    pattern = np.append(pattern[1:], index) #pattern의 0이 아닌 1부터 인덱스를 다음 인덱스로 옮겨 다음 100개로 다음 음표 예측 준비


In [ ]:
offset=0 #객체의 위치 표현
output_notes=[]
#모델에 의해 예측된 값을 바탕으로 노트와 코드 객체를 만든다
for pattern in prediction_output:
  #패턴(출력값)이 chord일때
  if('.'in pattern) or pattern.isdigit(): # '.'이 있거나 문자열에 숫자인 문자만 있을 경우
    notes_in_chord=pattern.split('.')
    notes=[]
    for current_note in notes_in_chord: #코드로 판별된 chord들
      new_note = note.Note(int(current_note)) #chord를 정수로 바꾼 후 note 객체로 바꿔준다.(코드를 구성하는 개별 note들)
      new_note.storedInstrument=instrument.Piano() #piano 형식으로 바꾼다
      notes.append(new_note) #note객체들을 notes 리스트에 추가
    new_chord=chord.Chord(notes) #notes에 있는 note들을 하나의 코드로 묶는다
    new_chord.offset=offset #new_chord 객체의 offset 속성을 현재의 offset으로 설정한다.-> 코드 객체의 타이밍 조절
    output_notes.append(new_chord) #최종 리스트에 코드 추가
  #패턴(출력값)이 note일때
  else:
    new_note=note.Note(pattern) #바로 객체 생성
    new_note.offset=offset #note 객체의 offset을 현재 offset으로 설정
    new_note.storeInstrument=instrument.Piano() #피아노로 변경
    output_notes.append(new_note) #최종 리스트에 note 객체 추가

  #각 반복마다 오프셋을 0.5씩 증가 -> 만약 증가시키지 않으면 같은 오프셋(위치)에 음이 쌓이게 된다
  offset+=0.5

In [ ]:
from music21 import stream

midi_stream=stream.Stream(output_notes) #하나의 Stream으로 리스트를 변경
midi_stream.write('midi', fp='test_output.mid') #midifile로 작성

'test_output.mid'